In [1]:
# Step 1 – Load datasets

# Import libraries
import pandas as pd
import numpy as np

# Define file paths
sales_path = "../data/raw/sales_data.csv"
features_path = "../data/raw/features_data.csv"
stores_path = "../data/raw/stores_data.csv"

# Load datasets with error handling
try:
    sales = pd.read_csv(sales_path)
    features = pd.read_csv(features_path)
    stores = pd.read_csv(stores_path)
    print("✅ Datasets successfully loaded.")
except Exception as e:
    raise FileNotFoundError(f"❌ Error loading files: {e}")



✅ Datasets successfully loaded.


In [57]:
# Step 2 – Data quality checks

# Quick dataset overview
for name, df in {"Sales": sales, "Features": features, "Stores": stores}.items():
    print(f"\n{name} shape: {df.shape}")
    display(df.head())


Sales shape: (421570, 5)


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False



Features shape: (8190, 12)


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False



Stores shape: (45, 3)


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [58]:
# Display basic information for all datasets
for name, df in {"Sales": sales, "Features": features, "Stores": stores}.items():
    print(f"\n📌 {name} dataset info:")
    df.info()


📌 Sales dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB

📌 Features dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkD

In [59]:
# Check missing values in all datasets
for name, df in {"Sales": sales, "Features": features, "Stores": stores}.items():
    print(f"\nMissing values in {name}:")
    display(df.isnull().sum())


Missing values in Sales:


Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64


Missing values in Features:


Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64


Missing values in Stores:


Store    0
Type     0
Size     0
dtype: int64

In [60]:
# Step 3 – Cleaning process

# Check and remove duplicates
for name, df in {"Sales": sales, "Features": features, "Stores": stores}.items():
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        df.drop_duplicates(inplace=True)
        print(f"⚠️ Removed {duplicates} duplicate rows from {name}")
    else:
        print(f"✅ No duplicates found in {name}")

✅ No duplicates found in Sales
✅ No duplicates found in Features
✅ No duplicates found in Stores


In [61]:
# Data type corrections
for df in [sales, features, stores]:
    if "Store" in df.columns:
        df["Store"] = df["Store"].astype(str)

In [62]:
# Convert 'Date' to datetime format
for name, df in {"Sales": sales, "Features": features}.items():
    df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    invalid_dates = df["Date"].isna().sum()
    if invalid_dates > 0:
        print(f"⚠️ {name}: {invalid_dates} invalid dates found")
        print(df.loc[df["Date"].isna(), "Date"].head())

⚠️ Sales: 253414 invalid dates found
2    NaT
3    NaT
6    NaT
7    NaT
10   NaT
Name: Date, dtype: datetime64[ns]
⚠️ Features: 4905 invalid dates found
2    NaT
3    NaT
6    NaT
7    NaT
10   NaT
Name: Date, dtype: datetime64[ns]


In [63]:
# Fill CPI and Unemployment per Store
for col in ["CPI", "Unemployment"]:
    if col in features.columns:
        features[col] = features.groupby("Store")[col].transform(lambda x: x.fillna(x.mean()))

# Fill MarkDown values with 0 (assume no discount)
markdown_cols = [col for col in features.columns if "MarkDown" in col]
features[markdown_cols] = features[markdown_cols].fillna(0)

In [64]:
# Step 4 – Merge and save

# Merge sales and features
sales_features = pd.merge(sales, features, on=["Store", "Date"], how="left")

# Merge with stores
df = pd.merge(sales_features, stores, on="Store", how="left")

In [65]:
# Remove rows with invalid dates
rows_before = df.shape[0]
df = df[df["Date"].notna()]
df.head()
print(f"✅ Removed {rows_before - df.shape[0]} rows with invalid dates")

✅ Removed 27622126 rows with invalid dates


In [68]:
# Save cleaned file 
output_path = "../data/clean/cleaned_data.csv"
df.to_csv(output_path, index=False)
print(f"✅ Cleaned dataset saved to {output_path}")


✅ Cleaned dataset saved to ../data/clean/cleaned_data.csv
